# 정적 크롤링
```
웹페이지  HTML 소스코드를 직접 가져와서 데이터를 추출
정적웹페이지는 서버에서 미리 준비된 html을 브라우져에 전송,  javascript로 동적으로 컨텐츠를 생성하지 않음

request : html소스 가져오기
BeatifulSoup : 파싱
조건 : 페이지가 로드될때 모든 데이터가 HTML에 포함
프로그램 실행속도 빠름.. 그러나 페이지 로딩속도는 느림

웹페이지의 크롤링 허용여부 : 웹사이트주소/robots.txt
모두허용
User-agent:*
Allow:/
or
User-agent:*
DisAllow:

모두 접근 금지
User-agent:*
DisAllow:/

```

In [ ]:
!pip install requests beautifulsoup4 -q

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# 1.웹 페이지 접속(요청)
url = 'https://www.naver.com/'
response = requests.get(url)
# 2. 상태 코드(200 성공) 404 페이지 찾을수 없음:url정보가 잘못되었을  500 번대 에러는 서버내부에러
if response.status_code == 200:
  # 3. HTML 파싱
  soup = BeautifulSoup(response.text, 'html.parser')
  span_text = soup.findAll('span')
  for span in span_text:
    print(span.text)
else:
  print(f'요청실패 : {response.status_code}')

상단영역 바로가기
서비스 메뉴 바로가기
새소식 블록 바로가기
쇼핑 블록 바로가기
관심사 블록 바로가기
MY 영역 바로가기
위젯 보드 바로가기
보기 설정 바로가기
    
검색
 입력도구 
입력도구
자동완성/최근검색어펼치기


<ipython-input-3-43ddbaede483>:8: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  span_text = soup.findAll('span')


# 연습용 html

In [ ]:
# request.get(url) 로 가져온 html이 다음과 같다고 가정
html = '''
<h1 id="title">크롤링 연습</h1><div class="top"><ul
class="menu"><li><a href=http://www.sample.co.kr/member/login.html
class="login loing2">로그인 </a></li></ul><ul class="brand"><li><a href="http://www/.
sample.co.kr/media/>사이트1<li><a href="http://www.sample.co.kr/
academy/">사이트2</a></li></ul></div>
'''
soup = BeautifulSoup(html,'html.parser')
print(soup.prettify())

<h1 id="title">
 크롤링 연습
</h1>
<div class="top">
 <ul class="menu">
  <li>
   <a class="login loing2" href="http://www.sample.co.kr/member/login.html">
    로그인
   </a>
  </li>
 </ul>
 <ul class="brand">
  <li>
   <a "="" academy="" href="http://www/.
sample.co.kr/media/&gt;사이트1&lt;li&gt;&lt;a href=" http:="" www.sample.co.kr="">
    사이트2
   </a>
  </li>
 </ul>
</div>



# 태그정보를 이용한 파싱
```
태그 <태그> ~~~~~ </태그>
```

In [ ]:
tag_h1 = soup.h1  # soup.find('h1')
tag_div = soup.div
tag_ul = soup.ul
tag_li = soup.li
tag_a = soup.a
print(tag_a)

<a class="login loing2" href="http://www.sample.co.kr/member/login.html">로그인 </a>


In [ ]:
# 지정된 태그를 모두  리스트형태로
tag_ul_all = soup.find_all('ul')
tag_li_all = soup.find_all('li')
tag_a_all = soup.find_all('a')
tag_a_all

[<a class="login loing2" href="http://www.sample.co.kr/member/login.html">로그인 </a>,
 <a "="" academy="" href="http://www/.
 sample.co.kr/media/&gt;사이트1&lt;li&gt;&lt;a href=" http:="" www.sample.co.kr="">사이트2</a>]

# 속성을 이용한 파싱
```
  attrs : 속성이름과 속성값의 dict형태
  find(): 속성을 이용해서 특정 태그 파싱
```

In [ ]:
print(f'태그정보를 파싱 : {tag_a}')
print(f'태그의 속성정보 : {tag_a.attrs}')
print(f'a 태그의 href 속성값 : {tag_a.attrs["href"]}')

태그정보를 파싱 : <a class="login loing2" href="http://www.sample.co.kr/member/login.html">로그인 </a>
태그의 속성정보 : {'href': 'http://www.sample.co.kr/member/login.html', 'class': ['login', 'loing2']}
a 태그의 href 속성값 : http://www.sample.co.kr/member/login.html


In [ ]:
soup.find('a',attrs = {'class':'login'})

<a class="login loing2" href="http://www.sample.co.kr/member/login.html">로그인 </a>

In [ ]:
title = soup.find(id='title')
title, title.string,  title.text

(<h1 id="title">크롤링 연습</h1>, '크롤링 연습', '크롤링 연습')

# select를 사용해서 지정한 태그를 모두 파싱

In [ ]:
li_lists = soup.select('div>ul>li')
for li in li_lists:
  print(li.string)

로그인 
사이트2


# 커피매장 정보 가져오기
```
https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=1
```

In [ ]:
# 매장정보를 추출하는 함수(태그객체들)
page_num = 1
def get_store_info(page_num):
  url = f'https://www.hollys.co.kr/store/korea/korStore2.do?pageNo={page_num}'
  # 1. 사이트에 접속해서 html 정보 가져오기
  response = requests.get(url)
  if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    # 원하는 정보를 포함하는 tag 위치 찾기
    tbody = soup.select('#contents > div.content > fieldset > fieldset > div.tableType01 > table > tbody > tr')
    isBreak  = tbody[0].find('td').string == '등록된 지점이 없습니다.'
  else:
    print(f'error : {response.status_code}')
  return tbody  ,isBreak

In [ ]:
# 전달된 정보(태그객체)에서 데이터 추출
def get_store(tbody,stores):
  keys = ['resion', 'store_name', 'status', 'address', 'phone_number']
  for i in range(len(tbody)):
    values = [td.string for idx, td in enumerate(tbody[i].select('td'))  if idx != 4]
    stores.append(dict(zip(keys,values)))

In [ ]:
from tqdm import tqdm
stores = []
for pagenum in tqdm(range(1,100)):
  tbody,isBreak = get_store_info(pagenum)
  if isBreak:
    break
  get_store(tbody,stores)

 49%|████▉     | 49/99 [01:44<01:46,  2.13s/it]


In [ ]:
print(f'매장개수 : {len(stores)}')
print(f'페이지수 : {len(stores)//10 + 1 if len(stores)%10 else 0}')

매장개수 : 483
페이지수 : 49


# 데이터베이스
```
mysql 접속
스키마 생성
테이블 생성(매장)
데이터 넣기
각종 조회해 보기
```

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 48.0 MB/s eta 0:00:00


In [ ]:
from mysql import connector
args = {
    "host" : "localhost",
    "user" : "root",
    "password" : "root1234",
    "port" : 3306
}
# 접속 여부 확인
try:
    conn = connector.connect(**args)
    cursor = conn.cursor()

    # 데이터 베이스 생성
    sql = "create database IF NOT EXISTS hollycoffe";
    cursor.execute(sql)

    # 데이터베이스 선택
    sql = 'use hollycoffe'
    cursor.execute(sql)

    # 테이블 생성
    sql = '''
        create table if not exists store(
            id int AUTO_INCREMENT primary key,
            resion varchar(100)
            ,store_name varchar(100)
            ,status varchar(100)
            ,address varchar(100)
            ,phone_number varchar(100)
        )
    '''
    cursor.execute(sql)
    # 테이터 삽입
    for s in stores:
        sql = 'insert into store values(null,%s,%s,%s,%s,%s)'
        # bs4.element.NavigableString --> str 타입으로 변경  : varchar 속성과 일치시키기 위해서
        data = [ str(d) for d in list(s.values()) ]
        cursor.execute(sql,   data  )
    # 모든작업이 끝나면 commit 해서 최종 반영
    conn.commit()
except Exception as e:
    print(f'error {e}')
else:
    print('정상처리 되었습니다.')
finally:  # 사용이 끝나면 자원 해제
    if cursor:
        cursor.close()
    if conn:
        conn.close()


error 2002: Can't connect to local MySQL server through socket '%-.100s' (%s) (Warning: not all arguments converted during string formatting)


NameError: name 'cursor' is not defined

# 다음뉴스 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

url = 'https://news.daum.net/?nil_profile=mini&nil_src=news'
response =  requests.get(url)
response.encoding = response.apparent_encoding  # 시스템에서 사용한 인코딩 자동 감지
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    li_lists = soup.select("#\\35 8d84141-b8dd-413c-9500-447b39ec29b9 > ul >li")

    news = []
    for i in tqdm(range(len(li_lists))):
      temp_dic = {}
      # print('제목 :',li_lists[i].select('strong.tit_txt')[0].text )
      temp_dic['제목'] = li_lists[i].select('strong.tit_txt')[0].text
      # if i == 0:
      #   print('내용 :',li_lists[i].select('p.desc_txt')[0].text )
      temp_dic['링크'] = li_lists[i].select('a')[0].attrs['href']
      # print('링크 :',li_lists[i].select('a')[0].attrs['href'])
      link = li_lists[i].select('a')[0].attrs['href']
      response = requests.get(link)
      response.encoding = response.apparent_encoding
      soup = BeautifulSoup(response.text, 'html.parser')
      ariticles = soup.select('#mArticle > div.news_view.fs_type1 > div.article_view > section > p')
      temp_dic['내용'] = ''.join([article.text for article in ariticles])
      news.append(temp_dic)
      # print('링크의 기사 확인 :')
      # for article in ariticles:
      #     print(article.text)

else:
    print('요청실패')


100%|██████████| 9/9 [00:09<00:00,  1.10s/it]


In [ ]:
news[:2]

[{'제목': '박상우 "토지거래허가구역 해제 집값 상승 영향, 필요시 적절한 조치"(종합)',
  '링크': 'https://v.daum.net/v/20250313151502400',
  '내용': '(서울=뉴스1) 김동규 임세원 윤주현 기자 = 박상우 국토교통부 장관이 13일 서울시의 토지거래허가구역 해제 이후 강남권을 중심으로 집값이 상승했다는 점을 인정하는가 하면, 필요시 서울시와 적절한 조치를 할 뜻을 밝혔다. 이날 국회에서 열린 국토교통위원회 전체회의에 참석한 박 장관은 최근 강남권 집값 상승과 토지거래허가구역 해제와의 연관성 질의에 이같이 답했다. 그는 "토지거래허가구역 해제에 대해 서울시의 자율성 보장 차원에서 적극적으로 반대하지는 않았다"면서도 "다만 앞으로 주택시장이 어떻게 변할지 서울시와 면밀하게 검토해 필요한 사안이 있으면 시기를 놓치지 않고 적절한 조치를 하겠다"고 말했다.이어 "지난해 여러 노력 결과 착공 물량이 상당히 많이 있는데, 앞으로도 공공에서 원활하게 공급이 잘 되도록 열심히 노력하겠다"고 덧붙였다.서울세종고속도로 안성구간 교량 붕괴사고와 관련해 재발방지 대책이 마련될 것으로 보인다. 박 장관은 "서울세종고속도로 건설현장 붕괴 사고는 현재 사고 원인 규명을 위해 사고조사위원회를 운영 중"이라며 "민간 전문가를 통해 사고 원인을 투명하고 객관적으로 조사하는 한편 유사 사고가 반복되지 않도록 철저한 재발 방지 대책을 마련하겠다"고 전했다.아울러 "부상자와 유가족분들이 사고의 아픔을 딛고 하루라도 빨리 일상으로 복귀할 수 있도록 장례와, 치료비, 생계지원, 법률자문 등을 충분히 하겠다"며 "인근 가옥에 대한 안전점검과, 주민 심리치료, 영업활동 보상 등 사고 현장 인근의 주민과 상인들을 위한 지원에도 최선을 다하겠다"고 말했다.사고 현장에 적용됐던 \'거더공법\'의 안전성을 다시 살펴볼 예정이다. 그는 "거더 장비가 길이 55m까지 설치 가능하다고 알고 있는데 공법 안전성이라든지, 유사 사고가 계속 발생하는 점을 감안해 특별히 관리

# API 사용
  - 네이버개발자 센터

In [ ]:
import os
import sys
import urllib.request
client_id = "sdNRbBL92SwJIst8YDmF"
client_secret = "WVBHSS_l0S"
encText = urllib.parse.quote("엔코아")
url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과
# url += '&display=100'
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()

import json

if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    # print(response_body.decode('utf-8'))
    json_dict = json.loads(response_body.decode('utf-8'))
    print(json_dict['items'][0]['title'])
    print(json_dict['items'][0]['link'])
    print(json_dict['items'][0]['description'])
    print(len(json_dict['items']))

else:
    print("Error Code:" + rescode)

<class 'str'>
SK네트웍스, 퀄컴 손잡고 AI 사업 강화
https://n.news.naver.com/mnews/article/015/0005104853?sid=101
자동차 종합관리 자회사 SK스피드메이트는 사고 차량 AI 자동 견적 시스템을 도입했으며, 데이터 솔루션 및 컨설팅 전문 자회사 <b>엔코아</b>는 AI 활용 기반이 되는 데이터 자산화 전략을 선보였다. 중고폰 거래 플랫폼... 
10


# 공공기관 API

In [ ]:
# Python3 샘플 코드 #


import requests

url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
service_key = '4dr0Ui4/QMen1/waNp8qkB24tz549lFOnJIJl/BzZami5c434Bui7bVOp5j1EVKBJzQF2VCgW8QY2lbbXyhqNA=='
params ={'serviceKey' : service_key, 'YM' : '201201', 'NAT_CD' : '112', 'ED_CD' : 'E' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>0000</resultCode><resultMsg>OK</resultMsg></header><body><items><item><ed>\xeb\xb0\xa9\xed\x95\x9c\xec\x99\xb8\xeb\x9e\x98\xea\xb4\x80\xea\xb4\x91\xea\xb0\x9d</ed><edCd>E</edCd><natCd>112</natCd><natKorNm>\xec\xa4\x91  \xea\xb5\xad</natKorNm><num>167022</num><rnum>1</rnum><ym>201201</ym></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>1</totalCount></body></response>'


In [ ]:
import xml.etree.ElementTree as ET
# 디코딩
xml_string = response.content.decode('utf-8')
# xml 파싱
root = ET.fromstring(xml_string)

# xml -> json 함수
def xml_to_json(element):
  json_dict = {}
  for child in element:
    if len(child) > 0:
      json_dict[child.tag] = xml_to_json(child)
    else:
      json_dict[child.tag] = child.text
  return json_dict
# xml형태를 json으로 적용
result = xml_to_json(root)
result

{'header': {'resultCode': '0000', 'resultMsg': 'OK'},
 'body': {'items': {'item': {'ed': '방한외래관광객',
    'edCd': 'E',
    'natCd': '112',
    'natKorNm': '중  국',
    'num': '167022',
    'rnum': '1',
    'ym': '201201'}},
  'numOfRows': '10',
  'pageNo': '1',
  'totalCount': '1'}}